In [68]:
import tensorflow as tf

In [69]:
import pandas as pd
import os
import numpy as np
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from datetime import datetime, timedelta
from sklearn.metrics import r2_score
import gzip
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import PredefinedSplit
import xgboost as xgb

import numpy as np
import pandas as pd
import datetime

from sklearn import model_selection

import tensorflow as tf
import keras as keras
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb


In [70]:
cd ..

/content/drive


In [71]:
cd content/drive/MyDrive

[Errno 2] No such file or directory: 'content/drive/MyDrive'
/content/drive


In [72]:
data_file = "ML_Project_data.parquet.gzip"
data = pd.read_parquet(data_file)
data.drop(columns=["BID","ASK","RET"], inplace=True)
data.info(verbose=True, show_counts=True)

FileNotFoundError: ignored

In [73]:
# normalize and rank every coloumn between [0,1]
data_ranked = data.drop(columns=['TICKER'])
data_ranked = data.rank(axis=0, pct=True)
rank_min = data_ranked.min(axis=0)
rank_max = data_ranked.max(axis=0)
data_mapping = ((2 * (data_ranked - rank_min) / (rank_max - rank_min)) - 1).round(2)
data_mapping.dropna(axis=1, inplace=True)
data_mapping['TICKER'] = data['TICKER']
data_mapping = data_mapping[data.columns]

In [74]:
data_mapping.drop(columns=['TICKER'], inplace=True)
data_mapping.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2492541 entries, (10057.0, Timestamp('1992-03-12 00:00:00')) to (93096.0, Timestamp('2022-12-30 00:00:00'))
Data columns (total 111 columns):
 #    Column                     Non-Null Count    Dtype  
---   ------                     --------------    -----  
 0    BIDLO                      2492541 non-null  float64
 1    ASKHI                      2492541 non-null  float64
 2    PRC                        2492541 non-null  float64
 3    VOL                        2492541 non-null  float64
 4    SHROUT                     2492541 non-null  float64
 5    OPENPRC                    2492541 non-null  float64
 6    RETX                       2492541 non-null  float64
 7    CAPEI                      2492541 non-null  float64
 8    bm                         2492541 non-null  float64
 9    evm                        2492541 non-null  float64
 10   pe_op_basic                2492541 non-null  float64
 11   pe_op_dil                  2492541 

In [75]:
# Check for multicollinearity or heteroskedasticity using Breusch Pagan Test

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

y = data_mapping['RETX']
X = data_mapping.drop(columns=['RETX'])
X = sm.add_constant(X)


model = sm.OLS(y, X)
results = model.fit()

_, p_value, _, _ = het_breuschpagan(results.resid, X)

print(f"P-value for Breusch-Pagan test: {p_value}")


P-value for Breusch-Pagan test: 0.0


In [76]:
# Removing heteroskedasticity or Varying Variance

y = data_mapping['RETX']
X = data_mapping.drop(columns=['RETX'])
X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit(cov_type='HC3')

robust_standard_errors = results.bse

print("Robust Standard Errors:")
print(robust_standard_errors)

p_values_after_transformation = results.pvalues

print("P-values after Huber-White transformation:")
print(p_values_after_transformation)

Robust Standard Errors:
const        0.000710
BIDLO        0.099237
ASKHI        0.104949
PRC          0.180230
VOL          0.000952
               ...   
fft          0.000359
GARCH_1_1    0.002532
GARCH_1_2    0.002992
GARCH_2_1    0.002638
GARCH_2_2    0.002282
Length: 111, dtype: float64
P-values after Huber-White transformation:
const         0.000000e+00
BIDLO         0.000000e+00
ASKHI         0.000000e+00
PRC           0.000000e+00
VOL          8.446520e-199
                 ...      
fft           7.524268e-02
GARCH_1_1     8.704951e-01
GARCH_1_2     5.276986e-03
GARCH_2_1     3.415360e-02
GARCH_2_2     9.963772e-01
Length: 111, dtype: float64


In [77]:
# Dataset after Huber White Transformation
data_mapping_transformed = data_mapping.copy()
data_mapping_transformed['RETX'] = results.fittedvalues
data_mapping_transformed.dropna(axis=1, inplace=True)
data_mapping_transformed.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2492541 entries, (10057.0, Timestamp('1992-03-12 00:00:00')) to (93096.0, Timestamp('2022-12-30 00:00:00'))
Data columns (total 111 columns):
 #    Column                     Non-Null Count    Dtype  
---   ------                     --------------    -----  
 0    BIDLO                      2492541 non-null  float64
 1    ASKHI                      2492541 non-null  float64
 2    PRC                        2492541 non-null  float64
 3    VOL                        2492541 non-null  float64
 4    SHROUT                     2492541 non-null  float64
 5    OPENPRC                    2492541 non-null  float64
 6    RETX                       2492541 non-null  float64
 7    CAPEI                      2492541 non-null  float64
 8    bm                         2492541 non-null  float64
 9    evm                        2492541 non-null  float64
 10   pe_op_basic                2492541 non-null  float64
 11   pe_op_dil                  2492541 

In [78]:
# Establishing window for training, validation and testing data

# first 8 years as training set
start_date_train_set = pd.Timestamp('1992-03-12')
end_date_train_set = pd.Timestamp('2000-03-12')

# following 3 years as validation set
end_date_validation_set = pd.Timestamp('2003-03-12')

# rest as test set
end_date_test_set = pd.Timestamp('2022-12-30')

training_set = data_mapping_transformed[
    (data_mapping_transformed.index.get_level_values('date') >= start_date_train_set) &
    (data_mapping_transformed.index.get_level_values('date') <= end_date_train_set)]

validation_set = data_mapping_transformed[
    (data_mapping_transformed.index.get_level_values('date') > end_date_train_set) &
    (data_mapping_transformed.index.get_level_values('date') <= end_date_validation_set)]

test_set = data_mapping_transformed[
    (data_mapping_transformed.index.get_level_values('date') > end_date_validation_set) &
    (data_mapping_transformed.index.get_level_values('date') <= end_date_test_set)]

In [79]:
def r_square_from_paper(y_true, y_pred):

    # Total Sum of Squares
    TSS = sum(y**2 for y in y_true)

    # Residual Sum of Squares (RSS)
    RSS = sum((y_true[i] - y_pred[i])**2 for i in range(len(y_true)))

    R_squared = 1 - (RSS/TSS)

    return R_squared

In [ ]:
# getting the best hyperparamater
best_alpha = None
best_l1_ratio = None
best_score = float('-inf')
test_r2 = float('-inf')
train_r2 = float('-inf')

# in last+1 iteration test_set will have 0 data -> end of leep
while len(test_set) > 0:
#for i in range(1,2):
    training_set.loc[:, 'test_fold'] = -1
    validation_set.loc[:, 'test_fold'] = 0

    # getting hyperparameters for test set only
    X_train = training_set.drop(columns=['RET'])
    Y_train = training_set['RET']

    # do elasticnetcv and train hyperparameters
    ElasticNetCV_model_test = ElasticNetCV()
    ElasticNetCV_model_test.fit(X_train, Y_train)

    alpha = ElasticNetCV_model_test.alpha_
    l1_ratio = ElasticNetCV_model_test.l1_ratio_
    score = ElasticNetCV_model_test.score(X_train, Y_train)

    # choosing the best parameters
    if score > best_score:
        best_alpha = alpha
        best_l1_ratio = l1_ratio
        best_score = score


    # tuning hyperparameters for both training and validation set
    training_and_validation_set = pd.concat([training_set, validation_set])
    testing_and_training_set_fold = np.array(training_and_validation_set['test_fold'])
    splits = PredefinedSplit(testing_and_training_set_fold)


    X_train_and_validation = training_and_validation_set.drop(columns=['RET'])
    Y_train_and_validation = training_and_validation_set['RET']

    # do elasticnetcv and train hyperparameters
    # cv -> cross-validation generator
    ElasticNetCV_mode_train_val = ElasticNetCV(cv=splits)
    ElasticNetCV_mode_train_val.fit(X_train_and_validation, Y_train_and_validation)


    alpha = ElasticNetCV_mode_train_val.alpha_
    l1_ratio = ElasticNetCV_mode_train_val.l1_ratio_
    score = ElasticNetCV_mode_train_val.score(X_train, Y_train)

    # choosing the best parameters with tuning along with validation set
    if score > best_score:
        best_alpha = alpha
        best_l1_ratio = l1_ratio
        best_score = score


    # predict values for returns
    test_set['test_fold'] = 1
    Y_pred = ElasticNetCV_mode_train_val.predict(test_set.drop(columns=['RET']))
    Y_true = test_set['RET']

    # calculate R^2
    r2_test = r_square_from_paper(Y_true.values, Y_pred)
    if r2_test > test_r2:
        test_r2 = r2_test


    r2_train = r_square_from_paper(Y_train_and_validation.values, ElasticNetCV_mode_train_val.predict(X_train_and_validation))
    if r2_train > train_r2:
        train_r2 = r2_train


    # rolling and recurssive window
    #print(f"start_date_train_set: {start_date_train_set}")
    start_date_train_set += pd.DateOffset(years=1)
    #print(f"end_date_train_set: {end_date_train_set}")
    end_date_train_set += pd.DateOffset(years=1)
    #print(f"end_date_test_set: {end_date_test_set}")
    end_date_test_set += pd.DateOffset(years=1)
    #print(f"end_date_validation_set: {end_date_validation_set}\n\n")
    end_date_validation_set += pd.DateOffset(years=1)


    training_set = pd.concat([training_set, data_mapping_transformed[
        (data_mapping_transformed.index.get_level_values('date') == end_date_train_set)
    ]])

    validation_set = data_mapping_transformed[
    (data_mapping_transformed.index.get_level_values('date') > end_date_train_set) &
    (data_mapping_transformed.index.get_level_values('date') <= end_date_validation_set)]


    test_set = data_mapping_transformed[
    (data_mapping_transformed.index.get_level_values('date') > end_date_validation_set) &
    (data_mapping_transformed.index.get_level_values('date') <= end_date_test_set)]


print(f"best_alpha: {best_alpha}")
print(f"best_l1_ratio: {best_l1_ratio}")
print(f"best_score: {best_score}")

In [ ]:
print("Best alpha:", best_alpha)
print("Best l1_ratio:", best_l1_ratio)
print("Best score:", best_score)
print(f"Train R^2: {train_r2}")
print(f"Test R^2: {test_r2}")

In [80]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [81]:
# LIGHTGBM for feature extraction
y = data_mapping_transformed['RETX']
X = data_mapping_transformed.drop(columns=['RETX'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = lgb.LGBMRegressor()
model.fit(X_train, y_train)


feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
sorted_importance = feature_importance.sort_values(by='Importance', ascending=False)


selected_features = sorted_importance[sorted_importance['Importance'] > 0]['Feature'].tolist()

X_selected = X[selected_features]

reduced_dataset = pd.concat([X_selected, y], axis=1)

print(sorted_importance)
print(selected_features)
print(len(selected_features))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.699209 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20313
[LightGBM] [Info] Number of data points in the train set: 1994032, number of used features: 110
[LightGBM] [Info] Start training from score -0.000128
            Feature  Importance
84       stochrsi_k         555
83         stochrsi         415
87     stoch_signal         390
86            stoch         341
85       stochrsi_d         256
..              ...         ...
35    capital_ratio           0
34   totdebt_invcap           0
33      debt_invcap           0
32    equity_invcap           0
109       GARCH_2_2           0

[110 rows x 2 columns]
['stochrsi_k', 'stochrsi', 'stoch_signal', 'stoch', 'stochrsi_d', 'rsi_14', 'eom', 'adx_neg', 'adx_pos', 'bollinger_pband', 'atr', 'tsi', 'OPENPRC', 'bollinger_lband_indicator', 'PRC', 'bollinger_hband_indicator', 'macd_diff', 'mfi', 'bollinger_

In [82]:
reduced_dataset.info(verbose=True, show_counts=True) # 35 features

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2492541 entries, (10057.0, Timestamp('1992-03-12 00:00:00')) to (93096.0, Timestamp('2022-12-30 00:00:00'))
Data columns (total 36 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   stochrsi_k                 2492541 non-null  float64
 1   stochrsi                   2492541 non-null  float64
 2   stoch_signal               2492541 non-null  float64
 3   stoch                      2492541 non-null  float64
 4   stochrsi_d                 2492541 non-null  float64
 5   rsi_14                     2492541 non-null  float64
 6   eom                        2492541 non-null  float64
 7   adx_neg                    2492541 non-null  float64
 8   adx_pos                    2492541 non-null  float64
 9   bollinger_pband            2492541 non-null  float64
 10  atr                        2492541 non-null  float64
 11  tsi                        2492541 non-null  float

In [ ]:
# Define the date ranges
start_date_train_set = reduced_dataset.index.get_level_values('date').min()
end_date_train_set = start_date_train_set + pd.DateOffset(years=8)
end_date_validation_set = end_date_train_set + pd.DateOffset(years=3)
end_date_test_set = reduced_dataset.index.get_level_values('date').max()

In [ ]:
best_alpha = None
best_l1_ratio = None
best_score = float('-inf')
test_r2 = float('-inf')
train_r2 = float('-inf')

while end_date_test_set <= reduced_dataset.index.get_level_values('date').max():

    training_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') >= start_date_train_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_train_set)
    ]

    validation_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') > end_date_train_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_validation_set)
    ]

    test_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') > end_date_validation_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_test_set)
    ]


    training_set['test_fold'] = -1
    validation_set['test_fold'] = 0


    X_train = training_set.drop(columns=['RET'])
    Y_train = training_set['RET']

    ElasticNetCV_model_test = ElasticNetCV()
    ElasticNetCV_model_test.fit(X_train, Y_train)

    alpha = ElasticNetCV_model_test.alpha_
    l1_ratio = ElasticNetCV_model_test.l1_ratio_
    score = ElasticNetCV_model_test.score(X_train, Y_train)

    if score > best_score:
        best_alpha = alpha
        best_l1_ratio = l1_ratio
        best_score = score


    training_and_validation_set = pd.concat([training_set, validation_set])
    testing_and_training_set_fold = np.array(training_and_validation_set['test_fold'])
    splits = PredefinedSplit(testing_and_training_set_fold)

    X_train_and_validation = training_and_validation_set.drop(columns=['RET'])
    Y_train_and_validation = training_and_validation_set['RET']

    ElasticNetCV_mode_train_val = ElasticNetCV(cv=splits)
    ElasticNetCV_mode_train_val.fit(X_train_and_validation, Y_train_and_validation)

    alpha = ElasticNetCV_mode_train_val.alpha_
    l1_ratio = ElasticNetCV_mode_train_val.l1_ratio_
    score = ElasticNetCV_mode_train_val.score(X_train, Y_train)

    if score > best_score:
        best_alpha = alpha
        best_l1_ratio = l1_ratio
        best_score = score


    test_set['test_fold'] = 1
    Y_pred = ElasticNetCV_mode_train_val.predict(test_set.drop(columns=['RET']))
    Y_true = test_set['RET']


    r2_test = r_square_from_paper(Y_true.values, Y_pred)
    if r2_test > test_r2:
        test_r2 = r2_test

    r2_train = r_square_from_paper(Y_train_and_validation.values, ElasticNetCV_mode_train_val.predict(X_train_and_validation))
    if r2_train > train_r2:
        train_r2 = r2_train

    start_date_train_set += pd.DateOffset(years=1)
    end_date_train_set += pd.DateOffset(years=1)
    end_date_test_set += pd.DateOffset(years=1)
    end_date_validation_set += pd.DateOffset(years=1)


    training_set = pd.concat([training_set, reduced_dataset[
        (reduced_dataset.index.get_level_values('date') == end_date_train_set)
    ]])
    validation_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') > end_date_train_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_validation_set)
    ]
    test_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') > end_date_validation_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_test_set)
    ]

In [ ]:
print("Best alpha:", best_alpha)
print("Best l1_ratio:", best_l1_ratio)
print("Best score:", best_score)
print(f"Train R^2: {train_r2}")
print(f"Test R^2: {test_r2}")

In [ ]:
pip install scikeras

In [83]:
import tensorflow as tf
from tensorflow import keras

In [84]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, MultiHeadAttention, LayerNormalization, Dropout
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [103]:
class Transformer(keras.Model):
    def __init__(self, encoder_layers, decoder_layers):
        super(Transformer, self).__init__()

        self.encoder = Encoder(encoder_layers)
        self.decoder = Decoder(decoder_layers)

    def call(self, inputs):
        encoder_output = self.encoder(inputs)
        decoder_output = self.decoder(encoder_output)

        return decoder_output

class Encoder(keras.Model):
    def __init__(self, num_layers):
        super(Encoder, self).__init__()

        self.layers1 = [keras.layers.Dense(64, activation='relu', name=f'dense_{i}') for i in range(num_layers)]
        self.dropout = keras.layers.Dropout(0.5)
        self.batch_norm = keras.layers.BatchNormalization()

    def call(self, inputs):
        for layer in self.layers1:
            inputs = layer(inputs)
        inputs = self.dropout(inputs)
        inputs = self.batch_norm(inputs)

        return inputs

class Decoder(keras.Model):
    def __init__(self, num_layers):
        super(Decoder, self).__init__()

        self.layers1 = [keras.layers.Dense(1, activation='linear', name=f'dense_{i}') for i in range(num_layers)]
        self.dropout = keras.layers.Dropout(0.5)
        self.batch_norm = keras.layers.BatchNormalization()

    def call(self, inputs):
        for layer in self.layers1:
            inputs = layer(inputs)
        inputs = self.dropout(inputs)
        inputs = self.batch_norm(inputs)

        return inputs

model = Transformer(6, 6)
model.compile(optimizer='adam', loss='mean_squared_error')


In [86]:
# first 8 years as training set
start_date_train_set = pd.Timestamp('1992-03-12')
end_date_train_set = pd.Timestamp('2000-03-12')

# following 3 years as validation set
end_date_validation_set = pd.Timestamp('2003-03-12')

# rest as test set
end_date_test_set = pd.Timestamp('2022-12-30')

In [118]:
training_set = reduced_dataset[
    (reduced_dataset.index.get_level_values('date') >= start_date_train_set) &
    (reduced_dataset.index.get_level_values('date') <= end_date_train_set)]

validation_set = reduced_dataset[
    (reduced_dataset.index.get_level_values('date') > end_date_train_set) &
    (reduced_dataset.index.get_level_values('date') <= end_date_validation_set)]

test_set = reduced_dataset[
    (reduced_dataset.index.get_level_values('date') > end_date_validation_set) &
    (reduced_dataset.index.get_level_values('date') <= end_date_test_set)]

In [ ]:
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
scaler = MinMaxScaler()
predicted_values_dict = {}
trading_signals = {}

while len(test_set) > 0:
    for permno, permno_data in training_set.groupby(level='PERMNO'):

        try:
          X_train = training_set.xs(permno, level='PERMNO').drop(columns=['RETX'])
          Y_train = training_set.xs(permno, level='PERMNO')['RETX']
          X_train = scaler.fit_transform(X_train)
          X_train = np.expand_dims(X_train, axis=1)

          try:
              X_validate = validation_set.xs(permno, level='PERMNO').drop(columns=['RETX'])
              Y_validate = validation_set.xs(permno, level='PERMNO')['RETX']
          except KeyError:
              print(f"Skipping PERMNO {permno} in validation_set.")
              continue

          X_validate = scaler.fit_transform(X_validate)
          X_validate = np.expand_dims(X_validate, axis=1)

          try:
              X_test = test_set.xs(permno, level='PERMNO').drop(columns=['RETX'])
              Y_test = test_set.xs(permno, level='PERMNO')['RETX']
          except KeyError:
              print(f"Skipping PERMNO {permno} in test_set.")
              continue

          X_test = scaler.fit_transform(X_test)
          X_test = np.expand_dims(X_test, axis=1)


          model.fit(X_train, Y_train, epochs=1, validation_data=(X_validate, Y_validate), callbacks=[early_stopping])

          predicted_values = model.predict(X_test)
          predicted_values_dict[permno] = predicted_values

          threshold = 0.01
          trading_signals[permno] = np.where(predicted_values > threshold, 1, np.where(predicted_values < -threshold, -1, 0))




          r2_test = r_square_from_paper(Y_test.values, predicted_values)
          print(f"R2 from paper for PERMNO {permno}: {r2_test}")

        except KeyError:
            continue

    # Rolling and recursive window
    start_date_train_set += pd.DateOffset(years=1)
    end_date_train_set += pd.DateOffset(years=1)
    end_date_test_set += pd.DateOffset(years=1)
    end_date_validation_set += pd.DateOffset(years=1)

    print("Update training, validation, and test sets")
    training_set = pd.concat([training_set, reduced_dataset[
        (reduced_dataset.index.get_level_values('date') == end_date_train_set)
    ]])

    validation_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') > end_date_train_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_validation_set)
    ]

    test_set = reduced_dataset[
        (reduced_dataset.index.get_level_values('date') > end_date_validation_set) &
        (reduced_dataset.index.get_level_values('date') <= end_date_test_set)
    ]


Streaming output truncated to the last 5000 lines.
54/54 [==============================] - 0s 4ms/step
R2 from paper for PERMNO 88992.0: [[0.71127427]]
7/7 [==============================] - 0s 4ms/step
R2 from paper for PERMNO 89130.0: [[0.3193568]]
133/133 [==============================] - 1s 4ms/step
R2 from paper for PERMNO 89195.0: [[0.6667827]]
133/133 [==============================] - 1s 4ms/step
R2 from paper for PERMNO 89269.0: [[0.68194]]
141/141 [==============================] - 1s 4ms/step
R2 from paper for PERMNO 91556.0: [[0.74737334]]
133/133 [==============================] - 1s 4ms/step
R2 from paper for PERMNO 92655.0: [[0.8165752]]
Update training, validation, and test sets
Skipping PERMNO 10057.0 in validation_set.
133/133 [==============================] - 1s 4ms/step
R2 from paper for PERMNO 10104.0: [[0.78050137]]
133/133 [==============================] - 1s 4ms/step
R2 from paper for PERMNO 10107.0: [[0.6548569]]
125/125 [==============================] - 1

In [121]:
model.summary()

Model: "transformer_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_11 (Encoder)        multiple                  23360     
                                                                 
 decoder_11 (Decoder)        multiple                  79        
                                                                 
Total params: 23439 (91.56 KB)
Trainable params: 23309 (91.05 KB)
Non-trainable params: 130 (520.00 Byte)
_________________________________________________________________


In [ ]:
trading_signals